# Introduction

In [1]:
import os

os.chdir('../')


The first building block in our system is the SignalSource

SignalSource is a class that has a __call__ method that returns the current value of that Signal. In this case the current mouse position

In [2]:
from genki_signals.signal_sources import MouseSignalSource

source = MouseSignalSource()

# TODO: Remove t argument in SignalSource 
source(1)

array([992.1796875 , 301.41796875])

But for time series applications we want to gather multiple samples of this signal.

For that we will need a Sampler.

A Sampler simply samples one or many SignalSources at a given sample rate.

### koma seinna?
Note: SignalSources can also be Samplers (they sample themselves and other sources at either a given sample rate or when data arrives)

In [7]:
from genki_signals.signal_sources import Sampler

# sources is a dictionary mapping source names to sources
#
# Note: if a SignalSource returns a dictionary then those names are used instead
sampler = Sampler(sources = {"mouse": source}, sample_rate=100)

# start gathering samples
sampler.start()

So now we continuously get samples of the mouse position.

# Explain better ?
And to get these samples out of the sampler we call sampler.read(), which returns all samples not previously read before.

In [14]:
print(sampler.read())
sampler.stop()

DataBuffer(max_size=None, data=timestamp: (768,)
mouse: (2, 768))


Now we might want to do some processing on these samples. 

That's where SignalFunctions come in.

SignalFunctions are functions that take in some signals (plural) and return a signal (singular).

In [15]:
import genki_signals.signal_functions as sf

# diff differentiates the mouse signals with regard to time and returns the signal "mouse_vel"
diff = sf.Differentiate(input_a="mouse", input_b="timestamp", name="mouse_vel")

But now we need to connect this SignalFunction to our mouse SignalSource.

For that we need SignalSystem.

SignalSystem takes in a Sampler/SignalSource and applies SignalFunctions on the samples

In [54]:
from genki_signals.signal_system import SignalSystem

system = SignalSystem(sampler, [diff])

# system.start() starts the internal sampler
system.start()

# Hard to explain well ?

SignalSystem also has a parameter called update_rate which defines how often we fetch samples from our Sampler.

We can then forward these samples to other places in our program, e.g. a buffer, a plotting class or saving to a file.

This is usefull since batched processing (saving a file / updating plot) is more efficient

In [17]:
# This is how we can stream data into a buffer

from genki_signals.buffers import DataBuffer

buffer = DataBuffer()

system.register_data_feed(id(buffer), lambda data: buffer.extend(data))

In [89]:
buffer

DataBuffer(max_size=None, data=timestamp: (18818,)
mouse: (2, 18820)
mouse_vel: (2, 18879))

Now everytime we call read we get the mouse_positions we have seen before and the mouse velocity at that time.

Now we might want to visualize our Signals to make sure they are how they are supposed to be or to see how they behave in different circumstances.

# TODO: continue with visualization
For that we can use ...

To train a model on these samples it is good practice to store them somewhere.

That brings us to recording.

# What if we don't call read(). then we try to save all samples at once (when stop_recording is called)
We just call system.start_recording(folder_path) which stores the signal samples in a file everytime system.read() is called.

In [39]:
system.start_recording("data/introduction_data/")

FileExistsError: [Errno 17] File exists: 'data/introduction_data'

and system.stop_recording() when we want to stop

In [40]:
system.stop_recording()
system.stop()

AttributeError: 'NoneType' object has no attribute 'stop'

Then we can load this session (stored data + metadata about recording) as follows:

In [11]:
from genki_signals.session import Session

session = Session.from_filename("data/introduction_data/")

session now includes our recorded data as well as metadata about the recording.

# Should Session have a toDataframe() method (data + signalFunctions)

In [23]:
session.data

DataBuffer(max_size=None, data=timestamp: (475,)
mouse: (2, 475))